In [1]:
import tabulate
import time

import torch

# Deep Learning Practical Session 1

EE-559

February 24, 2021

Austin Zadoks

## Multiple views of a storage

In [9]:
# 13x13 tensor filled with ones
m = torch.full((13, 13), 1, dtype=torch.int)

# Rows of twos starting with second row, every five rows
m[1::5, :] = 2
# Columns of twos starting with second column, every five columns
m[:, 1::5] = 2

# 2x2 areas of threes at intersections of (fourth and fifth) and (ninth and tenth) rows and columns
m[3:5, 3:5] = 3
m[3:5, 8:10] = 3
m[8:10, 3:5] = 3
m[8:10, 8:10] = 3

# Nice printing to match exercise statement
print(tabulate.tabulate(m, tablefmt='plain'))

1  2  1  1  1  1  2  1  1  1  1  2  1
2  2  2  2  2  2  2  2  2  2  2  2  2
1  2  1  1  1  1  2  1  1  1  1  2  1
1  2  1  3  3  1  2  1  3  3  1  2  1
1  2  1  3  3  1  2  1  3  3  1  2  1
1  2  1  1  1  1  2  1  1  1  1  2  1
2  2  2  2  2  2  2  2  2  2  2  2  2
1  2  1  1  1  1  2  1  1  1  1  2  1
1  2  1  3  3  1  2  1  3  3  1  2  1
1  2  1  3  3  1  2  1  3  3  1  2  1
1  2  1  1  1  1  2  1  1  1  1  2  1
2  2  2  2  2  2  2  2  2  2  2  2  2
1  2  1  1  1  1  2  1  1  1  1  2  1


## Eigendecomposition

In [10]:
size = 20

diag_m = torch.diag(torch.arange(1, size + 1, dtype=torch.float))
gauss_m = torch.empty((size, size)).normal_()
gauss_m_inv = torch.inverse(gauss_m)
mat_mul_result = torch.mm(torch.mm(gauss_m_inv, diag_m), gauss_m)
eig_result = torch.eig(mat_mul_result)

print(tabulate.tabulate(eig_result.eigenvalues, headers=('Re', 'Im'), tablefmt='simple'))

      Re    Im
--------  ----
20           0
 1           0
19           0
 1.99999     0
18           0
 3           0
 4           0
17           0
 5           0
16           0
 6.00001     0
 7           0
15           0
14           0
13           0
 8           0
12           0
11           0
10           0
 9           0


## Flops per second

In [11]:
size = 5000

# Generate matrices
m1 = torch.empty((size, size)).normal_()
m2 = torch.empty((size, size)).normal_()

# Record initial and final time for multiplication
t0 = time.perf_counter()
mm_result = torch.mm(m1, m2)
t1 = time.perf_counter()

# Matrix multipliciation takes (A_ncol * A_nrow * B_nrow) operations
n_ops = size ** 3
flops_s = n_ops / (t1 - t0)

print(f'{flops_s:0.6e} FLOPS per second')

1.530454e+11 FLOPS per second


## Playing with strides

In [13]:
def mul_row(m):
    result = m.clone()
    size = m.size()
    for i in range(size[0]):
        for j in range(size[1]):
            result[i, j] *= i + 1
    return result

def mul_row_fast(m):
    return m.T.mul(torch.arange(1, m.size()[0] + 1)).T
    
def mul_row_faster(m):
    row_factors = torch.arange(1, m.size()[0] + 1, dtype=torch.float).reshape(-1, 1)
    return m.mul(row_factors)

In [23]:
m = torch.full((1000, 400), 2.0)

print('`mul_row`')
mr = %time mul_row(m)
print(mr)

print('`mul_row_fast`')
mrf = %timeit mul_row_fast(m)

print('`mul_row_faster`')
mrfr = %timeit mul_row_faster(m)

# same_results = (torch.all(mr == mrf) and torch.all(mrf == mrfr)).item
# print(f'All results are the same: {same_results}')

`mul_row`
CPU times: user 7.33 s, sys: 42.2 ms, total: 7.37 s
Wall time: 7.42 s
tensor([[   2.,    2.,    2.,  ...,    2.,    2.,    2.],
        [   4.,    4.,    4.,  ...,    4.,    4.,    4.],
        [   6.,    6.,    6.,  ...,    6.,    6.,    6.],
        ...,
        [1996., 1996., 1996.,  ..., 1996., 1996., 1996.],
        [1998., 1998., 1998.,  ..., 1998., 1998., 1998.],
        [2000., 2000., 2000.,  ..., 2000., 2000., 2000.]])
`mul_row_fast`
98.3 µs ± 4.89 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
`mul_row_faster`
93.2 µs ± 4.91 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


TypeError: eq() received an invalid combination of arguments - got (NoneType), but expected one of:
 * (Tensor other)
      didn't match because some of the arguments have invalid types: (!NoneType!)
 * (Number other)
      didn't match because some of the arguments have invalid types: (!NoneType!)
